# Excel操作基礎編

## ファイルの作成

In [1]:
import openpyxl as xl

# 新規ワークブックを作る
wb = xl.Workbook()

# アクティブなワークシートを作る
ws = wb.active

ws["A1"] = "こんにちは"
wb.save("1_1_hello.xlsx")

## ファイルの読み込み

In [2]:
import openpyxl as xl
wb = xl.load_workbook("hello.xlsx")

# 先頭のワークシートを取り出す
ws = wb.worksheets[0]

# A1セルの値を取り出す
cell = ws["A1"]

print(cell.value)

こんにちは


## ファイルへの書き込み

In [3]:
import openpyxl as xl

wb = xl.Workbook()
ws = wb.active

# セル名への書き込み
ws["A1"] = "他者と比較するのではなく、過去の自分と比較する"

# 行列指定の書き込み
ws.cell(row=2,column=1,value="俺とおまえは意見が違うからお互いに存在価値があるんだ。")

cell = ws.cell(row=3,column=1)
cell.value = "勝つこととは、負けを恐れないことを意味する。"

wb.save("cell_write.xlsx")

### 連続で値を書き込む

In [4]:
import openpyxl as xl

wb = xl.Workbook()
ws = wb.active

for i in range(10):
    ws.cell(row = (i+1),column = 1,value =i)
    
wb.save("renzoku.xlsx")

### 九九の表を作成

In [5]:
import openpyxl as xl
import pandas as pd

wb = xl.Workbook()
ws = wb.active

for i in range(1,10):
    for j in range(1,10):
        ws.cell(row = i,column = j,value =i*j)

wb.save("9×9.xlsx")       
df = pd.read_excel("9×9.xlsx",header=None)
df

,0,1,2,3,4,5,6,7,8
0,1,2,3,4,5,6,7,8,9
1,2,4,6,8,10,12,14,16,18
2,3,6,9,12,15,18,21,24,27
3,4,8,12,16,20,24,28,32,36
4,5,10,15,20,25,30,35,40,45
5,6,12,18,24,30,36,42,48,54
6,7,14,21,28,35,42,49,56,63
7,8,16,24,32,40,48,56,64,72
8,9,18,27,36,45,54,63,72,81


### セル名で入力（cordinateプロパティ）

In [6]:
import openpyxl as xl
import pandas as pd

wb = xl.Workbook()
ws = wb.active

for i in range(1,101):
    for j in range(1,101):
        cell = ws.cell(row = i,column = j)
        
        # セル名が取得できるcordinateプロパティ
        cell.value = cell.coordinate
        
wb.save("test100.xlsx")          

## Excelシート自動生成のアイデア

### 年齢生年対応表

In [7]:
import openpyxl as xl
import datetime
import pandas as pd

wb = xl.Workbook()
ws = wb.active

thisyear = datetime.date.today().year

for i in range(80):
    age = i
    year = thisyear - i
    
    age_cell = ws.cell(i+1,1)
    age_cell.value = str(i) + "歳"
    
    year_cell = ws.cell(i+1,2)
    year_cell.value = str(year)+"年"
    
wb.save("agelist.xlsx")  
df = pd.read_excel("agelist.xlsx",header=None)
df.head(8)

,0,1
0,0歳,2023年
1,1歳,2022年
2,2歳,2021年
3,3歳,2020年
4,4歳,2019年
5,5歳,2018年
6,6歳,2017年
7,7歳,2016年


### 小学校入学年度を計算

In [8]:
import openpyxl as xl
import datetime

wb = xl.Workbook()
ws = wb.active

base_year = datetime.date.today().year - 20

for i in range(20):
    year = base_year + i
    s1 = year - 7 # 4/2以降に生まれた人
    s2 = year - 6 # 早生まれの人
    sf = f"{s1}年4/2～{s2}年4/1に生まれた人"
    
    ws.cell(row = i+1,column=1,value=str(year)+"年度")
    ws.cell(row = i+1,column=2,value=sf)
    
wb.save("入学list.xlsx") 

## Excelシートの連続読み込み

### for文で１つずつ範囲内のセルを得る

In [9]:
import openpyxl as xl

wb = xl.load_workbook("test100.xlsx")
ws = wb.active

for y in range(2,5):
    r = []
    for x in range(2,5):
        v = ws.cell(y,x).value
        r.append(v)
    print(r)

['B2', 'C2', 'D2']
['B3', 'C3', 'D3']
['B4', 'C4', 'D4']


### ワークシートの特定の範囲を取り出す方法

In [10]:
import openpyxl as xl

wb = xl.load_workbook("test100.xlsx")
ws = wb.active

# for row in ws["B2":"F4"]:
#     r = []
#     for cell in row:
#         r.append(cell.value)
#     print(r)
    
#リスト内包表記
for row in ws["B2":"F4"]:
    print([c.value for c in row])

['B2', 'C2', 'D2', 'E2', 'F2']
['B3', 'C3', 'D3', 'E3', 'F3']
['B4', 'C4', 'D4', 'E4', 'F4']


### iter_rowsで繰り返し指定範囲を取得

In [11]:
import openpyxl as xl

wb = xl.load_workbook("test100.xlsx")
ws = wb.active

it = ws.iter_rows(
        min_row = 2, min_col = 2, 
        max_row = 5, max_col = 4)

for row in it:
    r = []
    for cell in row:
        r.append(cell.value)
    print(r)

['B2', 'C2', 'D2']
['B3', 'C3', 'D3']
['B4', 'C4', 'D4']
['B5', 'C5', 'D5']


## Excelシート抽出テクニック

In [18]:
import openpyxl as xl
import pandas as pd
wb = xl.load_workbook(r"C:\Users\toneriver\Desktop\python学習\自動処理の教科書\sample\ch2\uriage.xlsx",data_only=True)
ws = wb.active

rows = ws["A2":"F9"]

for row in rows:
    print([c.value for c in row])

['日付', '購入者', '品目', '個数', '値段', '小計']
[datetime.datetime(2020, 3, 3, 0, 0), '鈴木太郎', 'バナナ', 3, 1000, 3000]
[datetime.datetime(2020, 3, 5, 0, 0), '伊藤次郎', 'リンゴ', 2, 1500, 3000]
[datetime.datetime(2020, 3, 5, 0, 0), '西川三郎', 'イチゴ', 2, 2000, 4000]
[datetime.datetime(2020, 3, 7, 0, 0), '山田史朗', 'バナナ', 8, 1000, 8000]
[datetime.datetime(2020, 3, 15, 0, 0), '伊藤次郎', 'リンゴ', 2, 1500, 3000]
[datetime.datetime(2020, 3, 16, 0, 0), '山田史朗', 'イチゴ', 5, 2000, 10000]
[datetime.datetime(2020, 3, 20, 0, 0), '山下五郎', 'マンゴー', 3, 1900, 5700]


## Excel間を何度も行き来してコピペ

In [44]:
import openpyxl as xl
import pandas as pd
wb = xl.load_workbook(r"C:\Users\toneriver\Desktop\python学習\自動処理の教科書\sample\ch2\all-customer.xlsx",data_only=True)

# 名簿のシートを選択
ws = wb["名簿"]

# 抜き出すカラム名を指定
columns = [["名前","住所","購入プラン"]]

df = pd.read_excel(r"C:\Users\toneriver\Desktop\python学習\自動処理の教科書\sample\ch2\all-customer.xlsx",header=1)
df

,名前,住所,購入プラン
0,古田 みき,名古屋市,A
1,小田 誉大,大阪府,A
2,石塚 晃,京都市,A
3,山中 武雄,横浜市,B
4,二宮 良知,川崎市,B
...,...,...,...
95,内山 英彦,さいたま市,D
96,尾崎 章二,長野市,D
97,堀江 聡,堺市,D
98,田中 勝,札幌市,D


In [46]:
df[df["住所"]=="横浜市"]

,名前,住所,購入プラン
3,山中 武雄,横浜市,B
14,児玉 次郎,横浜市,D
25,東 太陽,横浜市,B
36,小西 博,横浜市,D
47,篠崎 良純,横浜市,C
58,鶴田 俊大,横浜市,D
69,木村 一平,横浜市,C
80,三好 哲也,横浜市,A
91,小笠原 信,横浜市,C
